## Evaluating Summarization Accuracy on Claude Model in Bedrock

In this notebook we utilize Claude via Bedrock with the FMEval Library to test Summarization Accuracy with three metrics: meteor, rouge, and bert score.

Environment:

- Base Python 3.0 kernel
- Studio Notebook instance type: ml.c5.xlarge

### Setup

In [1]:
#
# Install the fmeval-*-py3-none-any.whl distribution.
#

#!rm -Rf ~/.cache/pip/*

#!pip3 install fmeval-0.1.0-py3-none-any.whl --upgrade --upgrade-strategy only-if-needed --force-reinstall
#!pip3 install boto3==1.28.65

In [ ]:
import boto3
import json
import os

# This is dependent on the hardware that you run the evaluation on. If the machine has enough memory you can increase this value or remove this environment variable. As this is a smaller dataset we set this value to 1 to circumvent any OOM issues.
os.environ["PARALLELIZATION_FACTOR"] = "1"

# Bedrock clients for model inference
bedrock = boto3.client(service_name='bedrock')
bedrock_runtime = boto3.client(service_name='bedrock-runtime')

In [ ]:
import glob

# Check for beta wheel and built-in dataset
if not glob.glob("fmeval-0.1.0-py3-none-any.whl"):
    print("ERROR - please make sure file exists: fmeval-0.1.0-py3-none-any.whl")

if not glob.glob("xsum_sample.jsonl"):
    print("ERROR - please make sure file exists: xsum_sample.jsonl")

### Sample Bedrock Inference

In [ ]:
import json

model_id = 'anthropic.claude-v2'
accept = "application/json"
contentType = "application/json"

# Ensure that your prompt is structured in the format that Claude expects as documented here: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html#model-parameters-claude-request-body
prompt_data = """Human: Who is Barack Obama?

Assistant:
"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})

response = bedrock_runtime.invoke_model(
    body=body, modelId=model_id, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
print(response_body.get("completion"))

### FMEval Setup

In [ ]:
from fmeval.data_loaders.data_config import DataConfig
from fmeval.model_runners.bedrock_model_runner import BedrockModelRunner
from fmeval.constants import MIME_TYPE_JSONLINES
from fmeval.eval_algorithms.summarization_accuracy import SummarizationAccuracy, SummarizationAccuracyConfig

#### Data Config Setup

In [ ]:
config = DataConfig(
    dataset_name="xsum_dataset",
    dataset_uri="xsum_sample.jsonl",
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="document",
    target_output_location="summary"
)

#### Model Runner Setup

In [ ]:
bedrock_model_runner = BedrockModelRunner(
    model_id=model_id,
    output='completion',
    content_template='{"prompt": $prompt, "max_tokens_to_sample": 500}'
)

### Run Evaluation

In [ ]:
eval_algo = SummarizationAccuracy(SummarizationAccuracyConfig())

In [ ]:
eval_output = eval_algo.evaluate(model=bedrock_model_runner, dataset_config=config, 
                                 prompt_template="Human: $feature\n\nAssistant:\n", save=True)

#### Parse Evaluation Report

In [ ]:
# parse report
print(json.dumps(eval_output, default=vars, indent=4))

In [ ]:
import pandas as pd

data = []
with open("/tmp/eval_results/summarization_accuracy_xsum_dataset.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df['meteor_score'] = df['scores'].apply(lambda x: x[0]['value'])
df['rouge_score'] = df['scores'].apply(lambda x: x[1]['value'])
df['bert_score'] = df['scores'].apply(lambda x: x[2]['value'])
df